In [3]:
!pip install munkres

In [4]:
from munkres import Munkres
import copy
inf = 999999

In [5]:
#Если не удовлетворяет правилу треугольника, делаем чтобы удовлетворяло
def floyd_warshall_algorithm(matrix):
    n = len(matrix)
    matrix_new = copy.deepcopy(matrix)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if (matrix_new[i][j] < inf):
                    matrix_new[i][j] = min(matrix_new[i][j], matrix_new[i][k]+matrix_new[k][j])
    return matrix_new

In [18]:
def Christofides_recurrent(input_matrix):
    matrix = floyd_warshall_algorithm(input_matrix)
    n = len(matrix)
    m = Munkres()            #Решаем задачу о назначениях
    idx = m.compute(matrix)
    way = dict(idx)
    #восстанавливаем циклы
    flags = [0]*n
    cycles = [[]]
    while sum(flags) < n:
        one_cycle = []
        i = flags.index(0)
        while flags[i] == 0:
            flags[i] = 1
            one_cycle.append(i)
            i = way[i]
        cycles.append(one_cycle)
    cycles = cycles[1:]
    if len(cycles) == 1:
        return cycles[0]
    #Объединяем циклы в точку и повторяем
    new_matrix = [[inf] * len(cycles) for i in range(len(cycles))]
    shortcuts = [[(-1, -1)] * len(cycles) for i in range(len(cycles))] 
    for i in range(len(cycles)):
        for j in range(len(cycles)):
            if(i != j):
                for p in range(len(cycles[i])):
                    for q in range(len(cycles[j])):
                        x = cycles[i][p]
                        y = cycles[j][q]
                        if(matrix[x][y] < new_matrix[i][j]):
                            new_matrix[i][j] = matrix[x][y]
                            shortcuts[i][j] = (x, y)
    external_cycle = Christofides_recurrent(new_matrix)
    #Объединяем циклы, выписывая порядки обхода малых циклов по порядку обхода большого
    external_cycle = external_cycle + external_cycle[0:2]
    perfect_cycle = []
    for i in range(1, len(external_cycle)-1):
        sequence_in = shortcuts[external_cycle[i-1]][external_cycle[i]][1]
        next_in = shortcuts[external_cycle[i]][external_cycle[i+1]][1]
        this_cycle = copy.deepcopy(cycles[external_cycle[i]])
        this_cycle = this_cycle + this_cycle[0:2]
        j = this_cycle[1:-1].index(sequence_in)+1
        direct_out = this_cycle[j-1]
        reverse_out = this_cycle[j+1]
        if(matrix[direct_out][next_in]-matrix[direct_out][sequence_in] <= matrix[reverse_out][next_in]-matrix[reverse_out][sequence_in]):
            perfect_cycle = perfect_cycle + this_cycle[j:-1]
            perfect_cycle = perfect_cycle + this_cycle[1:j]
        else:
            part_1 = this_cycle[1:j+1]
            part_1.reverse()
            part_2 = this_cycle[j+1:-1]
            part_2.reverse()
            perfect_cycle = perfect_cycle + part_1 + part_2
    return perfect_cycle

In [24]:
def algorithm_2opt(matrix, input_sequence):
    sequence = copy.deepcopy(input_sequence)
    flag_is_improved = 1
    while flag_is_improved == 1:
        flag_is_improved = 0
        for i in range(len(sequence)-2):
            for j in range(i+2,min(len(sequence)+i-1, len(sequence))):
                if (matrix[sequence[i]][sequence[i+1]] + matrix[sequence[j]][sequence[(j+1)%len(sequence)]] > matrix[sequence[i]][sequence[j]] + matrix[sequence[i+1]][sequence[(j+1)%len(sequence)]]):
                    rev_part = sequence[i+1:j+1]
                    rev_part.reverse()
                    sequence = sequence[:i+1] + rev_part + sequence[j+1:]
                    flag_is_improved = 1
    return sequence

In [42]:
def distance(matirix, sequence, print_route=False):
    _dst = 0
    for i in range(len(sequence)):
        if print_route:
            print(f"Ребро {sequence[i]} -> {sequence[i+1] if i != len(sequence)-1 else sequence[1]} длины {matirix[sequence[i]][sequence[i+1] if i != len(sequence)-1 else sequence[1]]}")
        _dst += matirix[sequence[i]][sequence[i+1] if i != len(sequence)-1 else sequence[1]]
    return _dst

In [25]:
#тестим
#input_matrix = [[inf,10,15,19], [10, inf, 12, 17], [15, 12, inf, 8], [19, 17, 8, inf]]
input_matrix = [
    [ inf,  10,  15,  11,   2,  55],
    [  10, inf,  16,  18,  21,  13],
    [  15,  16, inf,  39,  22,   3],
    [  11,  18,  39, inf,  28,  25],
    [   2,  21,  22,  28, inf,   2],
    [  55,  13,   3,  25,   2, inf]
]
input_matrix

[[999999, 10, 15, 11, 2, 55],
 [10, 999999, 16, 18, 21, 13],
 [15, 16, 999999, 39, 22, 3],
 [11, 18, 39, 999999, 28, 25],
 [2, 21, 22, 28, 999999, 2],
 [55, 13, 3, 25, 2, 999999]]

In [27]:
%%time
sequence = Christofides_recurrent(input_matrix)
sequence

CPU times: user 603 µs, sys: 0 ns, total: 603 µs
Wall time: 609 µs


[5, 2, 1, 3, 0, 4]

In [28]:
sequence_2opt = algorithm_2opt(input_matrix, sequence)
sequence_2opt

[5, 2, 1, 3, 0, 4]

In [33]:
%%time

# five

input_matrix = [
    [ inf,   3,   4,   2,   7],
    [   3, inf,   4,   6,   3],
    [   4,   4, inf,   5,   8],
    [   2,   6,   5, inf,   6],
    [   7,   3,   8,   6, inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)
sequence_2opt
# Исходное
# 1, 3, 2, 5, 4 -> 19
# Наше
# 2, 5, 1, 4, 3 -> 25
# 2-opt
# 2, 5, 4, 1, 3 -> 23

CPU times: user 482 µs, sys: 5 µs, total: 487 µs
Wall time: 491 µs


[1, 4, 3, 0, 2]

In [43]:
distance(input_matrix, sequence, print_route=True)

Ребро 1 -> 4 длины 3
Ребро 4 -> 0 длины 7
Ребро 0 -> 3 длины 2
Ребро 3 -> 2 длины 5
Ребро 2 -> 4 длины 8


25

In [44]:
distance(input_matrix, sequence_2opt, print_route=True)

Ребро 1 -> 4 длины 3
Ребро 4 -> 3 длины 6
Ребро 3 -> 0 длины 2
Ребро 0 -> 2 длины 4
Ребро 2 -> 4 длины 8


23

---

In [52]:
%%time

# P01

input_matrix = [
        [ inf,  29,  82,  46,  68,  52,  72,  42,  51,  55,  29,  74,  23,  72,  46],
        [  29, inf,  55,  46,  42,  43,  43,  23,  23,  31,  41,  51,  11,  52,  21],
        [  82,  55, inf,  68,  46,  55,  23,  43,  41,  29,  79,  21,  64,  31,  51],
        [  46,  46,  68, inf,  82,  15,  72,  31,  62,  42,  21,  51,  51,  43,  64],
        [  68,  42,  46,  82, inf,  74,  23,  52,  21,  46,  82,  58,  46,  65,  23],
        [  52,  43,  55,  15,  74, inf,  61,  23,  55,  31,  33,  37,  51,  29,  59],
        [  72,  43,  23,  72,  23,  61, inf,  42,  23,  31,  77,  37,  51,  46,  33],
        [  42,  23,  43,  31,  52,  23,  42, inf,  33,  15,  37,  33,  33,  31,  37],
        [  51,  23,  41,  62,  21,  55,  23,  33, inf,  29,  62,  46,  29,  51,  11],
        [  55,  31,  29,  42,  46,  31,  31,  15,  29, inf,  51,  21,  41,  23,  37],
        [  29,  41,  79,  21,  82,  33,  77,  37,  62,  51, inf,  65,  42,  59,  61],
        [  74,  51,  21,  51,  58,  37,  37,  33,  46,  21,  65, inf,  61,  11,  55],
        [  23,  11,  64,  51,  46,  51,  51,  33,  29,  41,  42,  61, inf,  62,  23],
        [  72,  52,  31,  43,  65,  29,  46,  31,  51,  23,  59,  11,  62, inf,  59],
        [  46,  21,  51,  64,  23,  59,  33,  37,  11,  37,  61,  55,  23,  59, inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)
[i+1 for i in sequence_2opt]

# Исходное
# 1, 13, 2, 15, 9, 5, 7, 3, 12, 14, 10, 8, 6, 4, 11 -> 291
# Наше
# 15, 9, 5, 7, 3, 12, 14, 10, 8, 11, 1, 4, 6, 2, 13 -> 358
# 2-opt
# 15, 9, 10, 8, 2, 13, 1, 11, 4, 6, 14, 12, 3, 7, 5 -> 305

CPU times: user 5.04 ms, sys: 251 µs, total: 5.3 ms
Wall time: 5.34 ms


[15, 9, 10, 8, 2, 13, 1, 11, 4, 6, 14, 12, 3, 7, 5]

In [50]:
distance(input_matrix, sequence)

358

In [51]:
distance(input_matrix, sequence_2opt)

305

---

In [57]:
%%time

# GR17

input_matrix = [
    [inf, 633, 257,  91, 412, 150,  80, 134, 259, 505, 353, 324,  70, 211, 268, 246, 121],
    [633, inf, 390, 661, 227, 488, 572, 530, 555, 289, 282, 638, 567, 466, 420, 745, 518],
    [257, 390, inf, 228, 169, 112, 196, 154, 372, 262, 110, 437, 191,  74,  53, 472, 142],
    [ 91, 661, 228, inf, 383, 120,  77, 105, 175, 476, 324, 240,  27, 182, 239, 237,  84],
    [412, 227, 169, 383, inf, 267, 351, 309, 338, 196,  61, 421, 346, 243, 199, 528, 297],
    [150, 488, 112, 120, 267, inf,  63,  34, 264, 360, 208, 329,  83, 105, 123, 364,  35],
    [ 80, 572, 196,  77, 351,  63, inf,  29, 232, 444, 292, 297,  47, 150, 207, 332,  29],
    [134, 530, 154, 105, 309,  34,  29, inf, 249, 402, 250, 314,  68, 108, 165, 349,  36],
    [259, 555, 372, 175, 338, 264, 232, 249, inf, 495, 352,  95, 189, 326, 383, 202, 236],
    [505, 289, 262, 476, 196, 360, 444, 402, 495, inf, 154, 578, 439, 336, 240, 685, 390],
    [353, 282, 110, 324,  61, 208, 292, 250, 352, 154, inf, 435, 287, 184, 140, 542, 238],
    [324, 638, 437, 240, 421, 329, 297, 314,  95, 578, 435, inf, 254, 391, 448, 157, 301],
    [ 70, 567, 191,  27, 346,  83,  47,  68, 189, 439, 287, 254, inf, 145, 202, 289,  55],
    [211, 466,  74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, inf,  57, 426,  96],
    [268, 420,  53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202,  57, inf, 483, 153],
    [246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, inf, 336],
    [121, 518, 142,  84, 297,  35,  29,  36, 236, 390, 238, 301,  55,  96, 153, 336, inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)

# Исходное
# 1, 4, 13, 7, 8, 6, 17, 14, 15, 3, 11, 10, 2, 5, 9, 12, 16 -> 2085
# Наше
# 8, 6, 14, 15, 3, 11, 5, 10, 2, 13, 1, 4, 9, 12, 16, 7, 17 -> 2456
# 2-opt
# 8, 6, 3, 11, 5, 2, 10, 15, 14, 17, 9, 12, 16, 4, 13, 1, 7 -> 2191

CPU times: user 8.87 ms, sys: 141 µs, total: 9.01 ms
Wall time: 9.1 ms


[8, 6, 3, 11, 5, 2, 10, 15, 14, 17, 9, 12, 16, 4, 13, 1, 7]

In [55]:
distance(input_matrix, sequence)

2456

In [56]:
distance(input_matrix, sequence_2opt)

2191

---

In [64]:
%%time

#FRI26

input_matrix = [
    [inf, 83, 93,129,133,139,151,169,135,114,110, 98, 99, 95, 81,152,159,181,172,185,147,157,185,220,127,181],
    [ 83,inf, 40, 53, 62, 64, 91,116, 93, 84, 95, 98, 89, 68, 67,127,156,175,152,165,160,180,223,268,179,197],
    [ 93, 40,inf, 42, 42, 49, 59, 81, 54, 44, 58, 64, 54, 31, 36, 86,117,135,112,125,124,147,193,241,157,161],
    [129, 53, 42,inf, 11, 11, 46, 72, 65, 70, 88,100, 89, 66, 76,102,142,156,127,139,155,180,228,278,197,190],
    [133, 62, 42, 11,inf,  9, 35, 61, 55, 62, 82, 95, 84, 62, 74, 93,133,146,117,128,148,173,222,272,194,182],
    [139, 64, 49, 11,  9,inf, 39, 65, 63, 71, 90,103, 92, 71, 82,100,141,153,124,135,156,181,230,280,202,190],
    [151, 91, 59, 46, 35, 39,inf, 26, 34, 52, 71, 88, 77, 63, 78, 66,110,119, 88, 98,130,156,206,257,188,160],
    [169,116, 81, 72, 61, 65, 26,inf, 37, 59, 75, 92, 83, 76, 91, 54, 98,103, 70, 78,122,148,198,250,188,148],
    [135, 93, 54, 65, 55, 63, 34, 37,inf, 22, 39, 56, 47, 40, 55, 37, 78, 91, 62, 74, 96,122,172,223,155,128],
    [114, 84, 44, 70, 62, 71, 52, 59, 22,inf, 20, 36, 26, 20, 34, 43, 74, 91, 68, 82, 86,111,160,210,136,121],
    [110, 95, 58, 88, 82, 90, 71, 75, 39, 20,inf, 18, 11, 27, 32, 42, 61, 80, 64, 77, 68, 92,140,190,116,103],
    [ 98, 98, 64,100, 95,103, 88, 92, 56, 36, 18,inf, 11, 34, 31, 56, 63, 85, 75, 87, 62, 83,129,178,100, 99],
    [ 99, 89, 54, 89, 84, 92, 77, 83, 47, 26, 11, 11,inf, 23, 24, 53, 68, 89, 74, 87, 71, 93,140,189,111,107],
    [ 95, 68, 31, 66, 62, 71, 63, 76, 40, 20, 27, 34, 23,inf, 15, 62, 87,106, 87,100, 93,116,163,212,132,130],
    [ 81, 67, 36, 76, 74, 82, 78, 91, 55, 34, 32, 31, 24, 15,inf, 73, 92,112, 96,109, 93,113,158,205,122,130],
    [152,127, 86,102, 93,100, 66, 54, 37, 43, 42, 56, 53, 62, 73,inf, 44, 54, 26, 39, 68, 94,144,196,139, 95],
    [159,156,117,142,133,141,110, 98, 78, 74, 61, 63, 68, 87, 92, 44,inf, 22, 34, 38, 30, 53,102,154,109, 51],
    [181,175,135,156,146,153,119,103, 91, 91, 80, 85, 89,106,112, 54, 22,inf, 33, 29, 46, 64,107,157,125, 51],
    [172,152,112,127,117,124, 88, 70, 62, 68, 64, 75, 74, 87, 96, 26, 34, 33,inf, 13, 63, 87,135,186,141, 81],
    [185,165,125,139,128,135, 98, 78, 74, 82, 77, 87, 87,100,109, 39, 38, 29, 13,inf, 68, 90,136,186,148, 79],
    [147,160,124,155,148,156,130,122, 96, 86, 68, 62, 71, 93, 93, 68, 30, 46, 63, 68,inf, 26, 77,128, 80, 37],
    [157,180,147,180,173,181,156,148,122,111, 92, 83, 93,116,113, 94, 53, 64, 87, 90, 26,inf, 50,102, 65, 27],
    [185,223,193,228,222,230,206,198,172,160,140,129,140,163,158,144,102,107,135,136, 77, 50,inf, 51, 64, 58],
    [220,268,241,278,272,280,257,250,223,210,190,178,189,212,205,196,154,157,186,186,128,102, 51,inf, 93,107],
    [127,179,157,197,194,202,188,188,155,136,116,100,111,132,122,139,109,125,141,148, 80, 65, 64, 93,inf, 90],
    [181,197,161,190,182,190,160,148,128,121,103, 99,107,130,130, 95, 51, 51, 81, 79, 37, 27, 58,107, 90,inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)

# Исходное
# 1, 25, 24, 23, 26, 22, 21, 17, 18, 20, 19, 16, 11, 12, 13, 15, 14, 10, 9, 8, 7, 5, 6, 4, 3, 2 -> 937
# Наше
# 23, 24, 25, 12, 13, 11, 10, 9, 14, 15, 3, 1, 2, 4, 6, 5, 7, 8, 16, 19, 20, 18, 17, 21, 22, 26 -> 1043
# 2-opt
# 23, 24, 25, 12, 13, 11, 9, 10, 14, 15, 3, 1, 2, 4, 6, 5, 7, 8, 16, 19, 20, 18, 17, 21, 22, 26 -> 1042

CPU times: user 30.3 ms, sys: 2.46 ms, total: 32.8 ms
Wall time: 35.5 ms


In [61]:
distance(input_matrix, sequence)

1043

In [62]:
distance(input_matrix, sequence_2opt)

1042

---

In [65]:
%%time

#DANTZIG42

input_matrix = [
    [inf,   8,  39,  37,  50,  61,  58,  59,  62,  81, 103, 108, 145, 181, 187, 161, 142, 174, 185, 164, 137, 117, 114,  85,  77,  87,  91, 105, 111,  91,  83,  89,  95,  74,  67,  74,  57,  45,  35,  29,   3,   5],
    [  8, inf,  45,  47,  49,  62,  60,  60,  66,  81, 107, 117, 149, 185, 191, 170, 146, 178, 186, 165, 139, 122, 118,  89,  80,  89,  93, 106, 113,  92,  85,  91,  97,  81,  69,  76,  59,  46,  37,  33,  11,  12],
    [ 39,  45, inf,   9,  21,  21,  16,  15,  20,  40,  62,  66, 104, 140, 146, 120, 101, 133, 142, 120,  94,  77,  73,  44,  36,  44,  48,  62,  69,  50,  42,  55,  64,  44,  42,  61,  46,  41,  35,  30,  41,  55],
    [ 37,  47,   9, inf,  15,  20,  17,  20,  25,  44,  67,  71, 108, 144, 150, 124, 104, 138, 143, 123,  96,  80,  78,  48,  40,  46,  50,  63,  71,  51,  43,  55,  63,  43,  41,  60,  41,  34,  26,  21,  37,  41],
    [ 50,  49,  21,  15, inf,  17,  18,  26,  31,  50,  72,  77, 114, 150, 156, 130, 111, 143, 140, 124,  94,  83,  84,  53,  46,  46,  48,  64,  66,  46,  38,  50,  56,  35,  31,  42,  25,  20,  18,  18,  47,  53],
    [ 61,  62,  21,  20,  17, inf,   6,  17,  22,  41,  63,  68, 106, 142, 142, 115,  97, 129, 130, 106,  80,  68,  69,  41,  34,  30,  34,  47,  51,  30,  22,  34,  42,  23,  25,  44,  30,  34,  34,  35,  57,  64],
    [ 58,  60,  16,  17,  18,   6, inf,  10,  15,  35,  57,  61,  99, 135, 137, 110,  91, 123, 126, 106,  78,  62,  63,  34,  27,  28,  32,  46,  53,  34,  26,  39,  49,  30,  32,  51,  36,  38,  36,  33,  55,  61],
    [ 59,  60,  15,  20,  26,  17,  10, inf,   5,  24,  46,  51,  88, 124, 130, 104,  85, 117, 124, 105,  77,  60,  57,  28,  19,  29,  33,  49,  56,  38,  32,  44,  56,  39,  41,  60,  47,  48,  46,  40,  58,  61],
    [ 62,  66,  20,  25,  31,  22,  15,   5, inf,  20,  41,  46,  84, 120, 125, 105,  86, 118, 128, 110,  84,  61,  59,  29,  21,  32,  36,  54,  61,  43,  36,  49,  60,  44,  46,  66,  52,  53,  51,  45,  63,  66],
    [ 81,  81,  40,  44,  50,  41,  35,  24,  20, inf,  23,  26,  63,  99, 105,  90,  75, 107, 118, 104,  77,  50,  48,  22,  14,  27,  30,  48,  57,  49,  51,  63,  75,  62,  64,  83,  71,  73,  70,  65,  83,  84],
    [103, 107,  62,  67,  72,  63,  57,  46,  41,  23, inf,  11,  49,  85,  90,  72,  51,  83,  93,  86,  56,  34,  28,  23,  29,  36,  34,  46,  59,  60,  63,  76,  86,  78,  83, 102,  93,  96,  93,  87, 105, 111],
    [108, 117,  66,  71,  77,  68,  61,  51,  46,  26,  11, inf,  40,  76,  81,  62,  59,  84, 101,  97,  64,  42,  36,  35,  40,  47,  45,  59,  71,  71,  75,  87,  97,  89,  90, 110,  98,  99,  97,  91, 109, 113],
    [145, 149, 104, 108, 114, 106,  99,  88,  84,  63,  49,  40, inf,  35,  41,  34,  29,  54,  72,  71,  65,  49,  43,  69,  77,  78,  77,  85,  96, 103, 106, 120, 126, 121, 130, 147, 136, 137, 134, 117, 147, 150],
    [181, 185, 140, 144, 150, 142, 135, 124, 120,  99,  85,  76,  35, inf,  10,  31,  53,  46,  69,  93,  90,  82,  77, 105, 114, 116, 115, 119, 130, 141, 142, 155, 160, 159, 164, 185, 172, 176, 171, 166, 186, 186],
    [187, 191, 146, 150, 156, 142, 137, 130, 125, 105,  90,  81,  41,  10, inf,  27,  48,  35,  58,  82,  87,  77,  72, 102, 111, 112, 110, 115, 126, 136, 140, 150, 155, 155, 160, 179, 172, 178, 176, 171, 188, 192],
    [161, 170, 120, 124, 130, 115, 110, 104, 105,  90,  72,  62,  34,  31,  27, inf,  21,  26,  58,  62,  58,  60,  45,  74,  84,  84,  83,  88,  98, 109, 112, 123, 128, 127, 133, 155, 148, 151, 151, 144, 164, 166],
    [142, 146, 101, 104, 111,  97,  91,  85,  86,  75,  51,  59,  29,  53,  48,  21, inf,  31,  43,  42,  36,  30,  27,  56,  64,  66,  63,  66,  75,  90,  93, 100, 104, 108, 114, 133, 126, 131, 129, 125, 144, 147],
    [174, 178, 133, 138, 143, 129, 123, 117, 118, 107,  83,  84,  54,  46,  35,  26,  31, inf,  26,  45,  68,  62,  59,  88,  96,  98,  97,  98,  98, 115, 126, 123, 128, 136, 146, 159, 158, 163, 161, 157, 176, 180],
    [185, 186, 142, 143, 140, 130, 126, 124, 128, 118,  93, 101,  72,  69,  58,  58,  43,  26, inf,  22,  50,  70,  69,  99, 107,  95,  91,  79,  85,  99, 108, 109, 113, 124, 134, 146, 147, 159, 163, 156, 182, 188],
    [164, 165, 120, 123, 124, 106, 106, 105, 110, 104,  86,  97,  71,  93,  82,  62,  42,  45,  22, inf,  30,  49,  55,  81,  87,  75,  72,  59,  62,  81,  88,  86,  90, 101, 111, 122, 124, 135, 139, 139, 161, 167],
    [137, 139,  94,  96,  94,  80,  78,  77,  84,  77,  56,  64,  65,  90,  87,  58,  36,  68,  50,  30, inf,  21,  27,  54,  60,  47,  44,  31,  38,  53,  60,  62,  67,  75,  85,  98, 121, 108, 118, 113, 134, 140],
    [117, 122,  77,  80,  83,  68,  62,  60,  61,  50,  34,  42,  49,  82,  77,  60,  30,  62,  70,  49,  21, inf,   5,  32,  40,  36,  32,  36,  47,  61,  64,  71,  76,  79,  84, 105,  97, 102, 102,  95, 119, 124],
    [114, 118,  73,  78,  84,  69,  63,  57,  59,  48,  28,  36,  43,  77,  72,  45,  27,  59,  69,  55,  27,   5, inf,  29,  37,  39,  36,  42,  53,  62,  66,  78,  82,  81,  86, 107,  99, 103, 101,  97, 116, 119],
    [ 85,  89,  44,  48,  53,  41,  34,  28,  29,  22,  23,  35,  69, 105, 102,  74,  56,  88,  99,  81,  54,  32,  29, inf,   8,  12,   9,  28,  39,  36,  39,  52,  62,  54,  59,  79,  71,  73,  71,  67,  86,  90],
    [ 77,  80,  36,  40,  46,  34,  27,  19,  21,  14,  29,  40,  77, 114, 111,  84,  64,  96, 107,  87,  60,  40,  37,   8, inf,  11,  15,  33,  42,  34,  36,  49,  59,  50,  52,  71,  65,  67,  65,  60,  78,  87],
    [ 87,  89,  44,  46,  46,  30,  28,  29,  32,  27,  36,  47,  78, 116, 112,  84,  66,  98,  95,  75,  47,  36,  39,  12,  11, inf,   3,  21,  29,  24,  27,  39,  49,  42,  47,  66,  59,  64,  65,  62,  84,  90],
    [ 91,  93,  48,  50,  48,  34,  32,  33,  36,  30,  34,  45,  77, 115, 110,  83,  63,  97,  91,  72,  44,  32,  36,   9,  15,   3, inf,  20,  30,  28,  31,  44,  53,  46,  51,  70,  63,  69,  70,  67,  88,  94],
    [105, 106,  62,  63,  64,  47,  46,  49,  54,  48,  46,  59,  85, 119, 115,  88,  66,  98,  79,  59,  31,  36,  42,  28,  33,  21,  20, inf,  12,  20,  28,  35,  40,  43,  53,  70,  67,  75,  84,  79, 101, 107],
    [111, 113,  69,  71,  66,  51,  53,  56,  61,  57,  59,  71,  96, 130, 126,  98,  75,  98,  85,  62,  38,  47,  53,  39,  42,  29,  30,  12, inf,  20,  28,  24,  29,  39,  49,  60,  62,  72,  78,  82, 108, 114],
    [ 91,  92,  50,  51,  46,  30,  34,  38,  43,  49,  60,  71, 103, 141, 136, 109,  90, 115,  99,  81,  53,  61,  62,  36,  34,  24,  28,  20,  20, inf,   8,  15,  25,  23,  32,  48,  46,  54,  58,  62,  88,  77],
    [ 83,  85,  42,  43,  38,  22,  26,  32,  36,  51,  63,  75, 106, 142, 140, 112,  93, 126, 108,  88,  60,  64,  66,  39,  36,  27,  31,  28,  28,   8, inf,  12,  23,  14,  24,  40,  38,  46,  50,  53,  80,  86],
    [ 89,  91,  55,  55,  50,  34,  39,  44,  49,  63,  76,  87, 120, 155, 150, 123, 100, 123, 109,  86,  62,  71,  78,  52,  49,  39,  44,  35,  24,  15,  12, inf,  11,  14,  24,  36,  37,  49,  56,  59,  86,  92],
    [ 95,  97,  64,  63,  56,  42,  49,  56,  60,  75,  86,  97, 126, 160, 155, 128, 104, 128, 113,  90,  67,  76,  82,  62,  59,  49,  53,  40,  29,  25,  23,  11, inf,  21,  30,  33,  43,  54,  62,  66,  92,  98],
    [ 74,  81,  44,  43,  35,  23,  30,  39,  44,  62,  78,  89, 121, 159, 155, 127, 108, 136, 124, 101,  75,  79,  81,  54,  50,  42,  46,  43,  39,  23,  14,  14,  21, inf,   9,  25,  23,  34,  41,  45,  71,  80],
    [ 67,  69,  42,  41,  31,  25,  32,  41,  46,  64,  83,  90, 130, 164, 160, 133, 114, 146, 134, 111,  85,  84,  86,  59,  52,  47,  51,  53,  49,  32,  24,  24,  30,   9, inf,  18,  13,  24,  32,  38,  64,  74],
    [ 74,  76,  61,  60,  42,  44,  51,  60,  66,  83, 102, 110, 147, 185, 179, 155, 133, 159, 146, 122,  98, 105, 107,  79,  71,  66,  70,  70,  60,  48,  40,  36,  33,  25,  18, inf,  17,  29,  38,  45,  71,  77],
    [ 57,  59,  46,  41,  25,  30,  36,  47,  52,  71,  93,  98, 136, 172, 172, 148, 126, 158, 147, 124, 121,  97,  99,  71,  65,  59,  63,  67,  62,  46,  38,  37,  43,  23,  13,  17, inf,  12,  21,  27,  54,  60],
    [ 45,  46,  41,  34,  20,  34,  38,  48,  53,  73,  96,  99, 137, 176, 178, 151, 131, 163, 159, 135, 108, 102, 103,  73,  67,  64,  69,  75,  72,  54,  46,  49,  54,  34,  24,  29,  12, inf,   9,  15,  41,  48],
    [ 35,  37,  35,  26,  18,  34,  36,  46,  51,  70,  93,  97, 134, 171, 176, 151, 129, 161, 163, 139, 118, 102, 101,  71,  65,  65,  70,  84,  78,  58,  50,  56,  62,  41,  32,  38,  21,   9, inf,   6,  32,  38],
    [ 29,  33,  30,  21,  18,  35,  33,  40,  45,  65,  87,  91, 117, 166, 171, 144, 125, 157, 156, 139, 113,  95,  97,  67,  60,  62,  67,  79,  82,  62,  53,  59,  66,  45,  38,  45,  27,  15,   6, inf,  25,  32],
    [  3,  11,  41,  37,  47,  57,  55,  58,  63,  83, 105, 109, 147, 186, 188, 164, 144, 176, 182, 161, 134, 119, 116,  86,  78,  84,  88, 101, 108,  88,  80,  86,  92,  71,  64,  71,  54,  41,  32,  25, inf,   6],
    [  5,  12,  55,  41,  53,  64,  61,  61,  66,  84, 111, 113, 150, 186, 192, 166, 147, 180, 188, 167, 140, 124, 119,  90,  87,  90,  94, 107, 114,  77,  86,  92,  98,  80,  74,  77,  60,  48,  38,  32,   6, inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)

# Исходное
# 699
# Наше
#  -> 831
# 2-opt
#  -> 780

CPU times: user 84.4 ms, sys: 1.14 ms, total: 85.6 ms
Wall time: 91.3 ms


In [66]:
distance(input_matrix, sequence)

831

In [67]:
distance(input_matrix, sequence_2opt)

780

---

In [68]:
%%time

#ATT48

input_matrix = [
    [ inf,4727,1205,6363,3657,3130,2414, 563, 463,5654,1713,1604,2368,2201,1290,1004,3833,2258,3419,2267,2957, 720,1700,5279,2578,6076,3465,2654,3625,3115,1574,3951,1748,2142,6755,2383,3306,1029,3530, 825,2188,4820,3489,1947,6835,1542,2379,3744],
    [4727, inf,3588,2012,1842,6977,6501,5187,5028,2327,4148,4723,3635,3125,4907,3930,7463,6338,7243,5105,4043,4022,3677,2863,3106,1850,7173,6630,1204,6814,6001,3447,5253,2656,3123,6274,7183,5622,3085,4564,2756,1591,7027,6186,3472,5461,4390,2088],
    [1205,3588, inf,5163,2458,3678,3071,1742,1444,4462,1184,1520,1498,1103,1501, 951,4298,2903,3967,2169,2209, 652, 828,4136,1518,4873,3954,3254,2446,3581,2441,2960,1966, 950,5564,2916,3878,2035,2482,1027,1395,3617,3891,2686,5661,2023,1867,2560],
    [6363,2012,5163, inf,2799,8064,7727,6878,6581,1402,5366,5946,4679,4378,6225,5709,8417,7578,8296,6135,4802,5707,4982,2322,4178, 320,8186,7800,2778,7859,7408,3763,6461,4223,1427,7451,8263,7131,3669,6011,4638,1681,7987,7502,1877,6758,5360,2844],
    [3657,1842,2458,2799, inf,5330,4946,4200,3824,2012,2573,3157,1924,1580,3427,3179,5749,4793,5577,3409,2223,3066,2185,1860,1401,2491,5486,5035, 894,5141,4611,1669,3677,1590,3113,4682,5533,4352,1252,3227,2426,1169,5313,4706,3241,3962,2651, 304],
    [3130,6977,3678,8064,5330, inf, 743,3209,2670,6929,2831,2266,3407,3854,2178,4076, 727, 881, 293,1930,3310,3672,3315,6199,3932,7745, 365, 482,5774, 261,1659,4513,1746,4431,7910, 769, 207,2225,4435,2681,5053,6384, 550,1224,7805,1670,2704,5230],
    [2414,6501,3071,7727,4946, 743, inf,2468,1952,6673,2380,1795,3051,3405,1604,3382,1469, 168,1020,1681,3110,2993,2827,6009,3552,7412,1104, 267,5300, 821, 916,4348,1270,3890,7698, 332, 900,1484,4185,2049,4415,6051,1219, 482,7635,1054,2432,4884],
    [ 563,5187,1742,6878,4200,3209,2468, inf, 718,6203,2241,2051,2920,2762,1687,1304,3932,2331,3487,2669,3487,1175,2260,5840,3141,6596,3563,2728,4120,3240,1559,4507,2082,2658,7304,2512,3364, 985,4091,1319,2544,5358,3632,1987,7391,1785,2879,4296],
    [ 463,5028,1444,6581,3824,2670,1952, 718, inf,5789,1602,1343,2330,2291, 970,1451,3376,1796,2959,1951,2835,1112,1725,5346,2628,6285,3007,2193,3889,2661,1122,3920,1372,2391,6883,1927,2845, 611,3543, 676,2590,4993,3039,1486,6934,1112,2196,3876],
    [5654,2327,4462,1402,2012,6929,6673,6203,5789, inf,4392,4947,3648,3501,5274,5183,7216,6535,7140,5022,3621,5077,4090, 922,3207,1131,7014,6714,2437,6707,6477,2476,5432,3599,1102,6376,7121,6284,2497,5160,4318, 937,6795,6507,1268,5773,4249,1914],
    [1713,4148,1184,5366,2573,2831,2380,2241,1602,4392, inf, 586, 766,1029, 883,2040,3353,2224,3100,1049,1246,1625, 503,3841,1196,5054,3042,2488,2945,2676,2087,2331,1114,1650,5459,2132,3037,1958,1997, 931,2513,3701,2923,2137,5459,1394, 711,2534],
    [1604,4723,1520,5946,3157,2266,1795,2051,1343,4947, 586, inf,1299,1612, 406,2208,2824,1639,2542, 694,1586,1767,1050,4357,1770,5633,2498,1907,3520,2128,1558,2778, 531,2171,6003,1552,2472,1538,2506, 791,2912,4277,2403,1564,5983, 827, 892,3109],
    [2368,3635,1498,4679,1924,3407,3051,2920,2330,3648, 766,1299, inf, 646,1642,2446,3840,2905,3655,1488, 730,2096, 697,3076, 533,4363,3567,3122,2453,3219,2842,1592,1791,1480,4706,2772,3610,2721,1232,1656,2550,3001,3403,2860,4697,2126, 756,1836],
    [2201,3125,1103,4378,1580,3854,3405,2762,2291,3501,1029,1612, 646, inf,1853,2026,4349,3247,4119,1997,1341,1753, 606,3078, 419,4070,4052,3517,1923,3690,3032,1866,2142, 838,4593,3161,4060,2788,1380,1663,1932,2736,3915,3138,4647,2395,1351,1592],
    [1290,4907,1501,6225,3427,2178,1604,1687, 970,5274, 883, 406,1642,1853, inf,2029,2803,1438,2466, 986,1987,1593,1253,4716,2072,5915,2454,1764,3710,2082,1204,3164, 497,2287,6342,1419,2379,1134,2867, 554,2885,4569,2405,1289,6338, 555,1297,3406],
    [1004,3930, 951,5709,3179,4076,3382,1304,1451,5183,2040,2208,2446,2026,2029, inf,4759,3220,4368,2900,3151, 442,1765,4960,2444,5443,4396,3610,2932,4034,2572,3891,2525,1590,6278,3313,4261,2033,3398,1476,1241,4287,4390,2928,6419,2428,2749,3337],
    [3833,7463,4298,8417,5749, 727,1469,3932,3376,7216,3353,2824,3840,4349,2803,4759, inf,1601, 477,2359,3617,4345,3851,6433,4372,8098, 370,1206,6267, 726,2384,4754,2335,4991,8148,1452, 609,2949,4752,3331,5687,6746, 437,1948,8005,2334,3098,5618],
    [2258,6338,2903,7578,4793, 881, 168,2331,1796,6535,2224,1639,2905,3247,1438,3220,1601, inf,1165,1563,2988,2829,2666,5882,3401,7263,1233, 399,5138, 923, 794,4227,1117,3724,7565, 286,1049,1348,4051,1881,4248,5903,1322, 355,7508, 887,2302,4736],
    [3419,7243,3967,8296,5577, 293,1020,3487,2959,7140,3100,2542,3655,4119,2466,4368, 477,1165, inf,2170,3520,3965,3588,6393,4183,7977, 202, 767,6041, 438,1932,4706,2027,4711,8107,1061, 132,2503,4652,2972,5344,6617, 486,1501,7989,1962,2939,5469],
    [2267,5105,2169,6135,3409,1930,1681,2669,1951,5022,1049, 694,1488,1997, 986,2900,2359,1563,2170, inf,1430,2460,1547,4333,2019,5817,2079,1694,3910,1733,1813,2668, 654,2694,6029,1366,2130,1991,2525,1474,3542,4455,1923,1641,5957,1071, 777,3302],
    [2957,4043,2209,4802,2223,3310,3110,3487,2835,3621,1246,1586, 730,1341,1987,3151,3617,2988,3520,1430, inf,2779,1387,2905,1062,4482,3398,3119,2922,3087,3115,1240,1953,2175,4607,2796,3501,3119,1136,2173,3268,3136,3189,3029,4527,2355, 711,2042],
    [ 720,4022, 652,5707,3066,3672,2993,1175,1112,5077,1625,1767,2096,1753,1593, 442,4345,2829,3965,2460,2779, inf,1401,4781,2166,5427,3984,3212,2946,3620,2224,3603,2089,1496,6178,2906,3861,1719,3132,1040,1479,4211,3969,2553,6290,2012,2336,3189],
    [1700,3677, 828,4982,2185,3315,2827,2260,1725,4090, 503,1050, 697, 606,1253,1765,3851,2666,3588,1547,1387,1401, inf,3621, 903,4675,3537,2954,2475,3169,2427,2254,1578,1148,5177,2598,3521,2194,1833,1074,2054,3340,3423,2541,5213,1801,1077,2190],
    [5279,2863,4136,2322,1860,6199,6009,5840,5346, 922,3841,4357,3076,3078,4716,4960,6433,5882,6393,4333,2905,4781,3621, inf,2718,2042,6254,6024,2569,5966,5913,1687,4807,3384,1716,5699,6384,5787,1852,4687,4285,1272,6022,5892,1629,5178,3581,1639],
    [2578,3106,1518,4178,1401,3932,3552,3141,2628,3207,1196,1770, 533, 419,2072,2444,4372,3401,4183,2019,1062,2166, 903,2718, inf,3864,4097,3635,1932,3748,3274,1448,2284,1164,4286,3283,4136,3086, 967,1973,2285,2507,3935,3331,4312,2589,1284,1340],
    [6076,1850,4873, 320,2491,7745,7412,6596,6285,1131,5054,5633,4363,4070,5915,5443,8098,7263,7977,5817,4482,5427,4675,2042,3864, inf,7866,7483,2515,7539,7101,3449,6146,3938,1375,7134,7944,6831,3349,5709,4397,1363,7667,7190,1798,6446,5041,2528],
    [3465,7173,3954,8186,5486, 365,1104,3563,3007,7014,3042,2498,3567,4052,2454,4396, 370,1233, 202,2079,3398,3984,3537,6254,4097,7866, inf, 839,5973, 374,2019,4569,1996,4669,7970,1085, 305,2581,4532,2976,5339,6509, 287,1581,7844,1974,2838,5369],
    [2654,6630,3254,7800,5035, 482, 267,2728,2193,6714,2488,1907,3122,3517,1764,3610,1206, 399, 767,1694,3119,3212,2954,6024,3635,7483, 839, inf,5427, 558,1181,4349,1377,4044,7723, 356, 653,1744,4218,2241,4614,6121, 955, 743,7644,1231,2465,4957],
    [3625,1204,2446,2778, 894,5774,5300,4120,3889,2437,2945,3520,2453,1923,3710,2932,6267,5138,6041,3910,2922,2946,2475,2569,1932,2515,5973,5427, inf,5612,4824,2550,4050,1498,3476,5071,5980,4470,2096,3388,1911,1501,5831,4994,3704,4264,3209,1196],
    [3115,6814,3581,7859,5141, 261, 821,3240,2661,6707,2676,2128,3219,3690,2082,4034, 726, 923, 438,1733,3087,3620,3169,5966,3748,7539, 374, 558,5612, inf,1716,4280,1624,4298,7679, 735, 420,2263,4216,2606,4967,6179, 400,1277,7567,1609,2501,5032],
    [1574,6001,2441,7408,4611,1659, 916,1559,1122,6477,2087,1558,2842,3032,1204,2572,2384, 794,1932,1813,3115,2224,2427,5913,3274,7101,2019,1181,4824,1716, inf,4330,1180,3346,7545,1023,1808, 578,4062,1438,3693,5763,2115, 440,7537, 763,2404,4603],
    [3951,3447,2960,3763,1669,4513,4348,4507,3920,2476,2331,2778,1592,1866,3164,3891,4754,4227,4706,2668,1240,3603,2254,1687,1448,3449,4569,4349,2550,4280,4330, inf,3184,2510,3402,4031,4698,4281, 533,3245,3612,2187,4339,4265,3296,3576,1941,1381],
    [1748,5253,1966,6461,3677,1746,1270,2082,1372,5432,1114, 531,1791,2142, 497,2525,2335,1117,2027, 654,1953,2089,1578,4807,2284,6146,1996,1377,4050,1624,1180,3184, inf,2685,6475,1022,1952,1341,2963,1050,3358,4787,1926,1086,6436, 422,1244,3619],
    [2142,2656, 950,4223,1590,4431,3890,2658,2391,3599,1650,2171,1480, 838,2287,1590,4991,3724,4711,2694,2175,1496,1148,3384,1164,3938,4669,4044,1498,4298,3346,2510,2685, inf,4697,3693,4636,2975,1981,1909,1124,2718,4565,3548,4830,2839,2140,1751],
    [6755,3123,5564,1427,3113,7910,7698,7304,6883,1102,5459,6003,4706,4593,6342,6278,8148,7565,8107,6029,4607,6178,5177,1716,4286,1375,7970,7723,3476,7679,7545,3402,6475,4697, inf,7393,8097,7370,3515,6249,5379,2001,7738,7556, 461,6829,5267,3013],
    [2383,6274,2916,7451,4682, 769, 332,2512,1927,6376,2132,1552,2772,3161,1419,3313,1452, 286,1061,1366,2796,2906,2598,5699,3283,7134,1085, 356,5071, 735,1023,4031,1022,3693,7393, inf, 965,1542,3883,1913,4286,5772,1121, 600,7322, 902,2128,4608],
    [3306,7183,3878,8263,5533, 207, 900,3364,2845,7121,3037,2472,3610,4060,2379,4261, 609,1049, 132,2130,3501,3861,3521,6384,4136,7944, 305, 653,5980, 420,1808,4698,1952,4636,8097, 965, inf,2380,4629,2877,5250,6583, 570,1380,7986,1866,2904,5432],
    [1029,5622,2035,7131,4352,2225,1484, 985, 611,6284,1958,1538,2721,2788,1134,2033,2949,1348,2503,1991,3119,1719,2194,5787,3086,6831,2581,1744,4470,2263, 578,4281,1341,2975,7370,1542,2380, inf,3952,1127,3197,5518,2658,1002,7395, 951,2429,4380],
    [3530,3085,2482,3669,1252,4435,4185,4091,3543,2497,1997,2506,1232,1380,2867,3398,4752,4051,4652,2525,1136,3132,1833,1852, 967,3349,4532,4218,2096,4216,4062, 533,2963,1981,3515,3883,4629,3952, inf,2873,3080,2012,4324,4046,3478,3328,1755,1000],
    [ 825,4564,1027,6011,3227,2681,2049,1319, 676,5160, 931, 791,1656,1663, 554,1476,3331,1881,2972,1474,2173,1040,1074,4687,1973,5709,2976,2241,3388,2606,1438,3245,1050,1909,6249,1913,2877,1127,2873, inf,2374,4392,2943,1659,6285,1012,1563,3254],
    [2188,2756,1395,4638,2426,5053,4415,2544,2590,4318,2513,2912,2550,1932,2885,1241,5687,4248,5344,3542,3268,1479,2054,4285,2285,4397,5339,4614,1911,4967,3693,3612,3358,1124,5379,4286,5250,3197,3080,2374, inf,3386,5284,3997,5585,3386,3125,2664],
    [4820,1591,3617,1681,1169,6384,6051,5358,4993, 937,3701,4277,3001,2736,4569,4287,6746,5903,6617,4455,3136,4211,3340,1272,2507,1363,6509,6121,1501,6179,5763,2187,4787,2718,2001,5772,6583,5518,2012,4392,3386, inf,6314,5837,2205,5095,3680,1169],
    [3489,7027,3891,7987,5313, 550,1219,3632,3039,6795,2923,2403,3403,3915,2405,4390, 437,1322, 486,1923,3189,3969,3423,6022,3935,7667, 287, 955,5831, 400,2115,4339,1926,4565,7738,1121, 570,2658,4324,2943,5284,6314, inf,1676,7603,1964,2662,5184],
    [1947,6186,2686,7502,4706,1224, 482,1987,1486,6507,2137,1564,2860,3138,1289,2928,1948, 355,1501,1641,3029,2553,2541,5892,3331,7190,1581, 743,4994,1277, 440,4265,1086,3548,7556, 600,1380,1002,4046,1659,3997,5837,1676, inf,7521, 744,2325,4670],
    [6835,3472,5661,1877,3241,7805,7635,7391,6934,1268,5459,5983,4697,4647,6338,6419,8005,7508,7989,5957,4527,6290,5213,1629,4312,1798,7844,7644,3704,7567,7537,3296,6436,4830, 461,7322,7986,7395,3478,6285,5585,2205,7603,7521, inf,6805,5208,3102],
    [1542,5461,2023,6758,3962,1670,1054,1785,1112,5773,1394, 827,2126,2395, 555,2428,2334, 887,1962,1071,2355,2012,1801,5178,2589,6446,1974,1231,4264,1609, 763,3576, 422,2839,6829, 902,1866, 951,3328,1012,3386,5095,1964, 744,6805, inf,1644,3928],
    [2379,4390,1867,5360,2651,2704,2432,2879,2196,4249, 711, 892, 756,1351,1297,2749,3098,2302,2939, 777, 711,2336,1077,3581,1284,5041,2838,2465,3209,2501,2404,1941,1244,2140,5267,2128,2904,2429,1755,1563,3125,3680,2662,2325,5208,1644, inf,2532],
    [3744,2088,2560,2844, 304,5230,4884,4296,3876,1914,2534,3109,1836,1592,3406,3337,5618,4736,5469,3302,2042,3189,2190,1639,1340,2528,5369,4957,1196,5032,4603,1381,3619,1751,3013,4608,5432,4380,1000,3254,2664,1169,5184,4670,3102,3928,2532, inf],
]

sequence = Christofides_recurrent(input_matrix)
sequence_2opt = algorithm_2opt(input_matrix, sequence)

# Исходное
# 33523
# Наше
#  -> 43984
# 2-opt
#  -> 36308

CPU times: user 164 ms, sys: 2.13 ms, total: 166 ms
Wall time: 180 ms


In [69]:
distance(input_matrix, sequence)

43984

In [71]:
distance(input_matrix, sequence_2opt)

36308

---

In [76]:
# Open input file
infile = open('xit1083.tsp.txt', 'r')
Dimension = 1083

# Read instance header
# Name = infile.readline().strip().split()[1] # NAME
# FileType = infile.readline().strip().split()[1] # TYPE
# Comment = infile.readline().strip().split()[1] # COMMENT
# Dimension = infile.readline().strip().split()[1] # DIMENSION
# EdgeWeightType = infile.readline().strip().split()[1] # EDGE_WEIGHT_TYPE
# infile.readline()

# Read node list
nodelist = []
N = int(Dimension)
for i in range(0, int(Dimension)):
    x,y = infile.readline().strip().split()[1:]
    nodelist.append([float(x), float(y)])

# Close input file
infile.close()

In [81]:
from math import sqrt

matrix = [[0] * 1083] * 1083

for i in range(1083):
    for j in range(1083):
        if i == j: matrix[i][j] = inf
        else: matrix[i][j] = sqrt( (nodelist[i][0] - nodelist[j][0])**2 + (nodelist[i][1] - nodelist[j][1])**2)

In [84]:
%%time

sequence = Christofides_recurrent(matrix)
sequence_2opt = algorithm_2opt(matrix, sequence)

ValueError: 295 is not in list

In [90]:
! pip install tsplib95

In [91]:
import tsplib95
problem = tsplib95.load('xit1083.tsp')

In [96]:
problem.get_nodes()

In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.0.3'

In [4]:
import numpy as np

In [5]:
np.__version__

'1.18.1'

In [6]:
import os
print(os.environ['PATH'])

/anaconda3/envs/choosegift_bot/bin:/anaconda3/condabin:/anaconda3/bin:/Library/Frameworks/Python.framework/Versions/3.6/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/share/dotnet:/opt/X11/bin:/Applications/Postgres.app/Contents/Versions/latest/bin
